In [11]:
from hashlib import sha256

In [12]:
def hash(m):
    hex_hash = sha256(str(m).encode("utf-8")).hexdigest()
    return int(hex_hash, 16)

In [13]:
class Signer:
    def __init__(self, F, g):
        self.F = F # Z_p
        self.g = g # elliptic curve generator
        
        self.pk = F.random_element()
        self.pubk = self.pk * g
        
    def get_pubk(self):
        return self.pubk
    
    def initiate_blind_signing(self):
        self.k = F.random_element()
        R_prime = self.k * self.g
        return R_prime
    
    def blind_sign(self, blind_message):
        return self.pk * blind_message + self.k

In [14]:
class User:
    def __init__(self, F, g):
        self.F = F # Z_p
        self.g = g # elliptic curve generator
        
        
    def blind_message(self, message, R_prime):
        
        # Generate some random values for blinding 
        self.a = F.random_element()
        self.b = F.random_element()
        
        # Calculate blinding factor using R_prime provided by signer 
        # Equal to (a * k + b) * g
        self.blinding_factor = self.a * R_prime + self.b * self.g
        blinding_factor_x = F(self.blinding_factor[0])
        
        # Calculate message hash 
        # We use the order of the field we are working in as the modulus for the hash
        m_hash = self.F(hash(message))
        
        # 1 = ~a * a
        blinded_message = (~self.a) * blinding_factor_x * m_hash
        return blinded_message
        
    def unblind_signature(self, blind_signature):
        return (self.blinding_factor, blind_signature * self.a + self.b)    

In [22]:
class Verifier:
    def __init__(self, F, g):
        self.F = F # Z_p
        self.g = g # elliptic curve generator
        
        
    def verify(self, message, signature, signer_pub_key):
        (blinding_factor, s) = signature
                
        return s * g == blinding_factor + F(blinding_factor[0]) * F(hash(message)) * signer_pub_key

In [2]:
# Ethereum elliptic curve
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
Fp = GF(p)
E = EllipticCurve(Fp, [a,b])
GX = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
GY = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
g = E(GX,GY)
F = GF(g.order())

In [10]:
float(p - g.order())

4.3242038656565966e+38

In [24]:
alice = Signer(F, g)
alice_pubk = alice.get_pubk()
print("Alice PubK is: {}", alice_pubk)

Alice PubK is: {} (108369852679600291225475973392280217393785341341256811480663505636778071006345 : 44533451297094849474866514314387423381878653175108865776515241860764910720964 : 1)


In [25]:
bob = User(F, g)
message = "noone knows who I am"

In [26]:
charly = Verifier(F, g)

In [27]:
# Alice provide randomness to blind the message
R_prime = alice.initiate_blind_signing()
# Bob blinds the message for signing 
blinded_message = bob.blind_message(message, R_prime)
# Alice signs the blinided message she can not read
blinded_signature = alice.blind_sign(blinded_message)
# Bob unblinds the signature
unblinded_signature = bob.unblind_signature(blinded_signature)

In [28]:
# Bob proves to Charly that Alice signed over his message
charly.verify(message, unblinded_signature, alice_pubk)

True